# Naive Bayes - weather to play

In [42]:
import pandas as pd

weather2play = pd.read_csv('weather-play-dataset.csv')

In [43]:
weather2play

,Outlook,Temp,Humidity,Windy,Class
0,sunny,75,70,yes,Play
1,sunny,80,90,yes,DontPlay
2,sunny,85,85,no,DontPlay
3,sunny,72,95,no,DontPlay
4,sunny,69,70,no,Play
5,overcast,72,90,yes,Play
6,overcast,83,78,no,Play
7,overcast,64,65,yes,Play
8,overcast,81,75,no,Play
9,rain,71,80,yes,DontPlay


## Data wrangling
Categorize temperature to _Low_, _Medium_, _High_

In [44]:
bins = pd.IntervalIndex.from_tuples([(0, 75), (75, 85),(85,100)])
interval_labels = ['Low','Medium','High']
#weather2play["HumidityGroup"] = pd.cut(weather2play['Humidity'], bins=interval, right=False, labels=interval_labels)
# doesn't work as expected due to the bug described here https://github.com/pandas-dev/pandas/issues/21233
interval = dict(zip(bins,interval_labels))
weather2play["Humidity"] = pd.cut(weather2play['Humidity'],bins).map(interval)
weather2play


,Outlook,Temp,Humidity,Windy,Class
0,sunny,75,Low,yes,Play
1,sunny,80,High,yes,DontPlay
2,sunny,85,Medium,no,DontPlay
3,sunny,72,High,no,DontPlay
4,sunny,69,Low,no,Play
5,overcast,72,High,yes,Play
6,overcast,83,Medium,no,Play
7,overcast,64,Low,yes,Play
8,overcast,81,Low,no,Play
9,rain,71,Medium,yes,DontPlay


In [45]:
weather2play.value_counts('Outlook')

Outlook
rain        5
sunny       5
overcast    4
dtype: int64

In [47]:
weather2play.value_counts('Class')

Class
Play        9
DontPlay    5
dtype: int64

In [48]:
weather2play.value_counts(['Class','Outlook'])

Class     Outlook 
Play      overcast    4
DontPlay  sunny       3
Play      rain        3
DontPlay  rain        2
Play      sunny       2
dtype: int64

In [49]:
weather2play.value_counts(['Outlook','Class'])

Outlook   Class   
overcast  Play        4
rain      Play        3
sunny     DontPlay    3
rain      DontPlay    2
sunny     Play        2
dtype: int64

In [102]:
sample_size=len(weather2play)

p['Class']=weather2play.value_counts('Class')/sample_size
print("## Probability of class")
print (p['Class'])
p['Outlook']=weather2play.value_counts('Outlook')/sample_size
print("## Probability of outlook")
print (p['Outlook'])
p['Humidity']=weather2play.value_counts('Humidity')/sample_size
print("## Probability of Humidity")
print (p['Humidity'])
p['Windy']=weather2play.value_counts('Windy')/sample_size
print("## Probability of Wind")
print (p['Windy'])

## Probability of class
Class
Play        0.642857
DontPlay    0.357143
dtype: float64
## Probability of outlook
Outlook
rain        0.357143
sunny       0.357143
overcast    0.285714
dtype: float64
## Probability of Humidity
Humidity
Low       0.357143
Medium    0.357143
High      0.285714
dtype: float64
## Probability of Wind
Windy
no     0.571429
yes    0.428571
dtype: float64


In [112]:
weather2play.value_counts(['Class','Outlook'])
p['Class']['Play']

0.6428571428571429

In [116]:
p_play_knowing_overcast= p['Outlook knowing Play']['overcast'] * p['Class']['Play'] / p['Outlook']['overcast']
print("P(Play|overcast) = P(overcast|play) * P(Play) / P(overcast)")
print(str(p_play_knowing_overcast)+" = "
      +str(p['Outlook knowing Play']['overcast'])
      +" * "+str(p['Class']['Play'])
      +" / "+str(p['Outlook']['overcast']))

P(Play|overcast) = P(overcast|play) * P(Play) / P(overcast)
1.0 = 0.44444444444444436 * 0.6428571428571429 / 0.2857142857142857


In [101]:
p_play_knowing_wind= p_wind_knowing_play['yes'] * p_class['Play'] / p_wind['yes']
print("P(Play|wind) = P(wind|play) * P(Play) / P(wind)")
print(str(p_play_knowing_wind)+" = "
      +str(p_wind_knowing_play['yes'])
      +" * "+str(p_class['Play'])
      +" / "+str(p_wind['yes']))

P(Play|wind) = P(wind|play) * P(Play) / P(wind)
0.5000000000000001 = 0.3333333333333333 * 0.6428571428571429 / 0.42857142857142855


In [135]:
def p_of_A_and_B(A,B):
    """
    P(A and B)
    
    >>> p_of_A_and_B(A,B)
    """
    # weather2play is global yeark!
    return weather2play.value_counts([A,B]) / len(weather2play)


def p_of_A_eq_a_knowing_B_eq_b (A,a,B,b):
    """
    simplified Naive Bayes classifier
    P(A=a|B=b) = P(B=b|A=a) * P(A=a) / P(B=b)
    
    p_of_A_eq_a_knowing_B_eq_b('Class','Play','Outlook','overcast')
    """
    # P(A and B)
    p[B+' and '+A]=p_of_A_and_B(A,B)
    # P(B|A=a) = P(A and B) / P(A)
    p[B+' knowing '+A+'='+a] = p[B+' and '+A][a] / p[A][a]
    # return P(A=a|B=b) = P(B=b|A=a) * P(A=a) / P(B=b)
    return p[B+' knowing '+A+'='+a][b] * p[A][a] / p[B][b]


def naive_bayes(matrix):
    """
    Naive Bayes classifier
    WIP
    naive_bayes(weather2play)
    """
    sample_size = len(matrix)
    for row in matrix:
        p[i]=matrix.value_counts(i)/sample_size
        p[i,j]
    return "coucou"
    
    

In [128]:
p_of_A_eq_a_knowing_B_eq_b('Class','Play','Outlook','overcast')

1.0

In [129]:
print("P(A=a|B=b) = P(B=b|A=a) * P(A=a) / P(B=b)")
p_of_A_eq_a_knowing_B_eq_b('Class','Play','Windy','yes')

P(A=a|B=b) = P(B=b|A=a) * P(A=a) / P(B=b)


0.5000000000000001

In [130]:
p

Outlook
rain                                                                   0.357143
sunny                                                                  0.357143
overcast                                                               0.285714
Class                         Class
Play        0.642857
DontPlay    0.35714...
Outlook                       Outlook
rain        0.357143
sunny       0.357...
Humidity                      Humidity
Low       0.357143
Medium    0.357143...
Windy                         Windy
no     0.571429
yes    0.428571
dtype: f...
Class and Outlook             Class     Outlook 
Play      overcast    0.285...
Outlook knowing Play          Outlook
overcast    0.444444
rain        0.333...
Outlook and Class             Class     Outlook 
Play      overcast    0.285...
Outlook knowing Class=Play    Outlook
overcast    0.444444
rain        0.333...
Windy and Class               Class     Windy
Play      no       0.428571
Do...
Windy knowing Class=Play      Wi

14

In [ ]:
import 